In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd "/content/gdrive/Shared drives/CIS 530 Project/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/Shared drives/CIS 530 Project


In [0]:
!pip install tensorflow==1.14

In [0]:
!pip install bert-serving-client
!pip install -U bert-serving-server[http]

     |████████████████████████████████| 71kB 3.4MB/s 
     |████████████████████████████████| 358kB 13.3MB/s 
  Created wheel for GPUtil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=fc755bb5ee998aa8582a2386c282bc5f85286a980b59618946fee345ac9421fc
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
  Created wheel for flask-compress: filename=Flask_Compress-1.5.0-cp36-none-any.whl size=5273 sha256=438081ee371ab3d1e8c8a9831e2635334d5e1f1ea3bf8c2a184f5a578c8f2843
  Stored in directory: /root/.cache/pip/wheels/f7/e9/e4/5afc286be7c87461375e33152558415dfeb0c8f5af3b50e742
Successfully built GPUtil flask-compress


In [0]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip uncased_L-12_H-768_A-12.zip

--2020-05-01 02:34:27--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.128, 2607:f8b0:400e:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M   129MB/s    in 3.0s    

2020-05-01 02:34:30 (129 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]

Archive:  uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


In [0]:
!nohup bert-serving-start -model_dir=./multi_cased_L-12_H-768_A-12 > out.file 2>&1 &

In [0]:
from bert_serving.client import BertClient

In [0]:
bc = BertClient()

In [0]:
# Returns the dictionary from the file with translations
def get_translation_dict(path):
  en_to_spanish = dict()

  with open(path, 'r') as f:
    for line in f:
        line = line.rstrip('\n').split('\t')
        en_word = line[0]
        es_word = line[1]
        if en_word.isupper(): en_to_spanish[en_word] = es_word.upper()
        elif len(en_word) > 0 and en_word[0].isupper(): en_to_spanish[en_word] = es_word.capitalize()
        else: en_to_spanish[en_word] = es_word

  return en_to_spanish

en_to_spanish = get_translation_dict("translations_umwe_eng.txt") 
dutch_to_spanish = get_translation_dict("translations_umwe_ned.txt")
#en_to_spanish_mono = get_translation_dict("translations_bi.txt")

In [0]:
import numpy as np
def get_indexed_word_embeddings():
  files=['Data/eng.train', 'Data/eng.testa', 'Data/eng.testb','Data/ned.train', 'Data/ned.testa', 'Data/ned.testb']
  word_to_index=dict()
  matrix = [] # words by embedding-dimension
  idx=0
  for file in files:
    with open(file, 'r') as f:
        for line in f:
          line = line.rstrip('\n')
          if line == "": continue

          word = line.split()[0]
          if word == "": continue
          org_word = word
          if file.startswith('Data/ned'):
            word = dutch_to_spanish.get(word)
          if file.startswith('Data/eng'):
            word = en_to_spanish.get(word)
          if word == "": 
            word = org_word #if translation not available eg. the word is a number use original word
          if word not in word_to_index:
            word_to_index[word] = idx
            embedding = np.squeeze(bc.encode([word]), axis=0)
            matrix.append(embedding)
            idx=idx+1
          if idx%100==0:
            print("total words added:",idx)
  return matrix, word_to_index

matrix, word_to_index = get_indexed_word_embeddings()  

In [0]:
import pickle
def save_obj(obj, name ):
  with open('obj/'+ name + '.pkl', 'wb') as f:
    pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
  with open('obj/' + name + '.pkl', 'rb') as f:
    return pickle.load(f)


In [0]:
save_obj(mbert_matrix_v3, 'mbert_matrix_v3')
save_obj(mbert_word_to_index_v3, 'mbert_word_to_index_v3')

In [0]:
import numpy as np
mbert_matrix_v3 = load_obj('mbert_matrix_v3')
mbert_word_to_index_v3 = load_obj('mbert_word_to_index_v3')
print(len(mbert_matrix_v3))
print(len(mbert_word_to_index_v3))

34727
34727


In [0]:
import numpy as np
files=['Data/esp.train', 'Data/esp.testa', 'Data/esp.testb']

idx=len(mbert_matrix_v3)
for file in files:
  print('doing',file)
  with open(file, 'r') as f:
      for line in f:
        line = line.rstrip('\n')
        if line == "": continue

        word = line.split()[0]
        if word == "": continue
        org_word = word
        if file.startswith('Data/ned'):
          word = dutch_to_spanish.get(word)
        if file.startswith('Data/eng'):
          word = en_to_spanish.get(word)
        if word == "": 
          word = org_word #if translation not available eg. the word is a number use original word
        if word not in mbert_word_to_index_v3:
          mbert_word_to_index_v3[word] = idx
          embedding = np.squeeze(bc.encode([word]), axis=0)
          mbert_matrix_v3.append(embedding)
          idx=idx+1
        if idx%100==0:
          print("total words added:",idx)


Exception ignored in: <bound method Context.__del__ of <zmq.sugar.context.Context object at 0x7f1834113fc0>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/zmq/sugar/context.py", line 50, in __del__
    self.term()
  File "zmq/backend/cython/context.pyx", line 91, in zmq.backend.cython.context.Context.term
  File "zmq/backend/cython/checkrc.pxd", line 13, in zmq.backend.cython.checkrc._check_rc
KeyboardInterrupt


 42800
total words added: 42800
total words added: 42800
total words added: 42800
total words added: 42800
total words added: 42800
total words added: 42800
total words added: 42800
total words added: 42800
total words added: 42900
total words added: 42900
total words added: 42900
total words added: 42900
total words added: 42900
total words added: 42900
total words added: 42900
total words added: 42900
total words added: 42900
total words added: 42900
total words added: 42900
total words added: 42900
total words added: 42900
total words added: 42900
total words added: 42900
total words added: 42900
total words added: 42900
total words added: 43000
total words added: 43000
total words added: 43000
total words added: 43000
total words added: 43000
total words added: 43000
total words added: 43000
total words added: 43000
total words added: 43100
total words added: 43100
total words added: 43100
total words added: 43100
total words added: 43100
total words added: 43100
total words added:

In [0]:
print(len(mbert_matrix_v3))
print(len(mbert_word_to_index_v3))

56007
56007


In [0]:
save_obj(mbert_matrix_v3, 'mbert_matrix_v3')
save_obj(mbert_word_to_index_v3, 'mbert_word_to_index_v3')